# Setting up the libraries

In [ ]:
Pkg.add("LinearMaps")
Pkg.clone("https://github.com/Jutho/TensorOperations.jl")
Pkg.update()

# Exact diagonalization of the Ising model

In [2]:
using LinearMaps

In [3]:
# Ising model with transverse magnetic field h (critical h=1 by default)
function build_ising_ham(h=1.0)
    X = [0 1; 1 0]
    Z = [1 0; 0 -1]
    I2 = eye(2)
    XX = kron(X, X)
    ZI = kron(Z, I2)
    IZ = kron(I2, Z)
    H = -(XX + h/2*(ZI+IZ))
    return H
end


function apply_spin_hamiltonian{T}(v::AbstractVector{T}, ham::Matrix{T})
    D = length(v)
    N = convert(Int64, log2(D))
    tensor_shp = tuple(fill(2, N)...)  # tensor_shp =(2,2,2, ..., 2)
    cyclperm = tuple(2:N..., 1)  #tuple for cyclic permutation cyclperm = (2, 3, ... N, 1)
    out = zeros(tensor_shp)
    for n=1:N
        v, out = reshape(v, (4, 2^(N-2))), reshape(out, (4, 2^(N-2)))
        out += ham*v
        v, out = reshape(v, tensor_shp), reshape(out, tensor_shp)
        v, out = permutedims(v, cyclperm), permutedims(out, cyclperm) 
    end
    out = reshape(out, D)
    return out
end


N = 12
ising_h = build_ising_ham()
apply_ising_hamiltonian(v) = apply_spin_hamiltonian(v, ising_h)
H = LinearMap(apply_ising_hamiltonian, 2^N)
@time u, v = eigs(H; nev=1, which=:SR)  #:SR stands for smallest real part
energy = real(u[1])
e_density = energy/N
infN_e_density = -4/pi
info(e_density)
info((e_density - infN_e_density)/infN_e_density)

 12.928407 seconds (63.52 M allocations: 1.906 GiB, 2.51% gc time)


INFO: -1.276882929256731
INFO: 0.002861507511790359


# Uniform Matrix Product States

In [3]:
include("MPSTools.jl")
using MPSTools

In [4]:
d = 2
D = 5
rumps = rand_UMPS(Complex128, d, D)

info(bond_dim(rumps))
info(phys_dim(rumps))
info(mps_tensor(rumps))

INFO: 5
INFO: 2
INFO: Complex{Float64}[0.744652-1.12976im -0.968165-0.224989im; -1.22516-0.957561im 0.316324+0.637425im; -0.110278+0.552591im 0.43988+0.18355im; 0.956878+0.393327im 1.4291-0.0377493im; 0.142396-0.110386im 0.956832-0.911607im]

Complex{Float64}[1.54723+0.701967im -0.310647-0.71533im; -0.381559+0.255508im -0.132735+1.08072im; -0.370723-0.133146im 0.271987-0.238011im; -0.399657+1.31718im 0.632122+0.182089im; 0.275616+0.35622im -0.640325+1.31677im]

Complex{Float64}[-0.509559-0.258974im 0.57498+0.685549im; -1.0083+0.047313im 0.783889-0.0143781im; 0.0950174+1.32624im 0.306357-0.883913im; -0.251599+0.261489im 1.4577-0.792693im; -0.915025+0.00288091im -0.146286-0.369911im]

Complex{Float64}[0.540806+0.734083im 0.426209+0.286974im; 0.286103-0.262267im 0.497595-0.853037im; -0.156529+0.214838im -0.0508923-0.163969im; 0.787602+0.151493im -0.116621+0.941736im; -0.85885-0.0187503im -0.0986024+0.328805im]

Complex{Float64}[-0.663715-0.979546im 2.13778-0.0764858im; 0.750549+0.150234im

# How to use TensorOperations
https://github.com/Jutho/TensorOperations.jl

In [2]:
using TensorOperations

In [3]:
D = 8
d = 2
A=randn(D, d, D)
B=randn(D, d, D)
x=randn(D, D)
@tensor y[a, d] := (A[a, s, b] * x[b, c]) * conj(B[d, s, c])

8×8 Array{Float64,2}:
  2.8389      -2.36066  -16.2216   …    2.40719    2.27099   -0.064616
  0.48603    -11.1316   -15.7828       37.0245    -9.38914   -0.62949 
 -2.27375    -24.9968    -5.87084      20.6241   -10.3537     7.42278 
 -0.0387244   -4.10959    8.99863     -13.6481     1.58622   -7.43649 
  1.82327     -1.17272  -20.2554       -1.44551    5.08714  -11.6752  
 -0.601702     9.13717    9.18234  …  -19.6208     3.51833    4.48183 
  0.49232    -12.3988     8.97994       7.19677    6.28984   -9.78335 
 -2.70671     -5.77765    7.57405     -18.3485     5.09289    8.7062  